In [ ]:
import os
import random
from tifffile import imread, imwrite

def split_tiff(tiff_path, train_dir, test_dir, indices, is_label=False):
    # Read the TIFF file
    tiff_data = imread(tiff_path)

    # Check if the TIFF file contains multiple slices
    if len(tiff_data.shape) != 3:
        print("The TIFF file does not contain multiple slices.")
        return

    # Create the output directories if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Initialize counters for train and test slices
    train_counter = 0
    test_counter = 0

    # Save the slices into the appropriate directories
    for i, slice_data in enumerate(tiff_data):
        if i in indices['test']:
            if is_label:
                slice_filename = f"image_{test_counter+1:03d}.tif"  # No channel suffix for labels
            else:
                slice_filename = f"image_{test_counter+1:03d}_0000.tif"  # Channel suffix for images
            slice_path = os.path.join(test_dir, slice_filename)
            print(f"Slice {i+1} saved as {slice_path} (Test)")
            test_counter += 1
        else:
            if is_label:
                slice_filename = f"image_{train_counter+1:03d}.tif"  # No channel suffix for labels
            else:
                slice_filename = f"image_{train_counter+1:03d}_0000.tif"  # Channel suffix for images
            slice_path = os.path.join(train_dir, slice_filename)
            print(f"Slice {i+1} saved as {slice_path} (Train)")
            train_counter += 1

        imwrite(slice_path, slice_data)

    print("Slicing complete.")

def determine_split_indices(total_slices, test_ratio=0.2):
    indices = list(range(total_slices))
    random.shuffle(indices)
    split_point = int(total_slices * test_ratio)
    return {'test': indices[:split_point], 'train': indices[split_point:]}

# Specify the path to your image and label TIFF files
image_tiff_path = r"/alsxdata/ItamarAndDafna/Final_Project_rawdata/15_30_HIGH_5_LOW 3CM/M1/Interpolated 640 M1 stack.tif"
label_tiff_path = r"/alsxdata/ItamarAndDafna/Final_Project_rawdata/15_30_HIGH_5_LOW 3CM/M1/initial mask substack.tif"

# Specify the output directories for training and testing
image_train_dir = r"/alsxdata/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset024_1530640M1/imagesTr"
image_test_dir = r"/alsxdata/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset024_1530640M1/imagesTs"
label_train_dir = r"/alsxdata/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset024_1530640M1/labelsTr"
label_test_dir = r"/alsxdata/ItamarAndDafna/nnUNet/nnUNet_raw/Dataset024_1530640M1/labelsTs"  # Ground truth information for comparison later

# Read the image TIFF file to determine the number of slices
tiff_data = imread(image_tiff_path)
total_slices = tiff_data.shape[0]

# Determine the split indices
indices = determine_split_indices(total_slices, test_ratio=0.2)

# Call the function to split the image TIFF file into individual slices (with channel suffix)
split_tiff(image_tiff_path, image_train_dir, image_test_dir, indices)

# Call the function to split the label TIFF file into individual slices (without channel suffix)
split_tiff(label_tiff_path, label_train_dir, label_test_dir, indices, is_label=True)